In [1]:
#importing necessary libraries
from transformers import AutoProcessor, FlavaModel, FlavaImageModel, AutoImageProcessor, FlavaFeatureExtractor
import numpy as np
from torchmultimodal.models.flava.model import flava_model_for_classification
from transformers import FlavaProcessor, FlavaForPreTraining
import torchvision
import torch
import torch.nn as nn
import wandb

from tqdm import tqdm
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.datasets import MNIST, CUB200

from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
import torchvision.transforms as transforms
from torch.utils.data import random_split

/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from copy import deepcopy




# Define a function to compute the CKA
def cka(kernel1, kernel2):
    
    # Compute CKA
    cka_num = np.linalg.norm((kernel1 * kernel2))
    cka_den1 = np.linalg.norm((kernel1 * kernel1))
    cka_den2 = np.linalg.norm((kernel2 * kernel2))
    cka_value = cka_num/(cka_den1*cka_den2)



    return cka_value

# Check if a GPU is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained models
image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
model1 = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to(device) #Frozen model
model2 = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to(device) #partly-frozen model



# Freeze some layers (e.g., the first 3 layers)
for param in model1.parameters():
    param.requires_grad = False


child_counter = 0
for child in model2.children():   
    if child_counter < 2: #2 for 2 child frozen which gives best results as of now
        for param in child.parameters():
            param.requires_grad = False
        child_counter += 1



# Define a function to extract activations from a specific layer
def get_activations(model, x, layer_name):
    layers = list(model.children())
    activations = []
    for i, layer in enumerate(layers):
        outpts = model(**x)
        features = outpts.last_hidden_state  #.view(outpts.last_hidden_state.size(0), -1) 
        if i == layer_name:
            return features
        activations.append(features)
    return features

# Prepare data 
resize_transform = transforms.Resize((224, 224))
grayscale_transform = transforms.Grayscale(num_output_channels=3)
data_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()
    
])


cifar100_dataset = CIFAR100(root='./data/cifar100', train=True, transform=data_transform, download=True)
dataloader = DataLoader(cifar100_dataset, batch_size=1, shuffle=True)

# Extract representations from a specific layer
layer_name = 2  # Example layer number

for data in dataloader:
    inputs, _ = data
    
    inputs = image_processor(images=inputs, return_tensors="pt").to("cuda")
    outputs1 = model1(**inputs)
    outputs2 = model2(**inputs)

    
    representations1 = get_activations(model1, inputs, layer_name).detach().cpu().numpy()
    representations2 = get_activations(model2, inputs, layer_name).detach().cpu().numpy()
    break  # Process one batch (you can modify this loop to process more data)


representations1 = np.squeeze(representations1, axis=0)
representations2 = np.squeeze(representations2, axis=0)


# Calculate the CKA score
cka_score = cka(representations1, representations2)
print("CKA Score:", cka_score)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Files already downloaded and verified
CKA Score: 0.46880094210306805
